In [11]:
email_str = ""  # linkedin email
pass_str = ""  # linkedin password

In [12]:
driverPath = "chromedriver.exe"

In [13]:
Segments = ["About", "Activity", "Experience", "Skills", "Education", "Interests"]  # "Companies","Groups","Schools"

In [ ]:
login_link = "https://linkedin.com/uas/login"

# args
urls = ["https://www.linkedin.com/in/username/", "https://www.linkedin.com/in/username/", \
        "https://www.linkedin.com/in/username", "https://www.linkedin.com/in/username/"]

In [2]:
import os
import re
import time
import math
import random
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from timeit import default_timer
from numpy import unique, ravel
from numpy import sqrt, dot, array, diagonal, mean, transpose, eye, diag, ones
from numpy import transpose, diag, dot
from numpy.linalg import svd, inv, qr, det
from sklearn.linear_model import LinearRegression
from matplotlib.pyplot import figure
import seaborn as sns

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [5]:
def filter(lst):  # list of ugly strings
    ret = []
    for s in lst:
        s2 = ""
        add = True
        for l in s:
            if add:
                s2 += l
            if l == "<":
                add = False
            if not add and l == ">":
                add = True
        s2 = re.sub("<|\n", "", s2)
        s2 = re.sub("( )+", " ", s2)
        if len(s2) > 3:
            ret.append(s2)
    return ret

def deleteRepeats(rawData):
    dd = {seg: [] for seg in Segments}
    activeSegment = ""
    seen = []
    for row in rawData:
        for item in list(unique(row)):
            if item in Segments:
                activeSegment = item
            if activeSegment and item not in seen:
                seen.append(item)
                dd[activeSegment].append(item)
    return dd

In [6]:
def view(_mid_data_dd):
    for seg in _mid_data_dd:
        print("\n", seg.upper()+":")
        print(_mid_data_dd[seg], "\n")

def editSpan(_span):
    raw_data = []
    for i in range(len(_span)):
        if "<!-- -->" in str(_span[i]):
            txt = str(_span[i]).split("<!-- -->")
            raw_data.append(filter(txt))

    mid_data_dd = deleteRepeats(raw_data)
    
    view(mid_data_dd)  #### stable
    
    return mid_data_dd

# The LinkedIn Webscrapper Class  

Skills is currently broken

In [62]:



class LinkedOut(object):
    
    def logIn(self):
        # find em
        username = self.driver.find_element(By.ID, 'username')
        pword = self.driver.find_element(By.ID, 'password')
        # enter em
        pword.send_keys(pass_str)  ####
        username.send_keys(email_str)  ####
        self.driver.find_element(By.XPATH, "//button[@type='submit']").click()
        
        
    
    def __init__(self, links, driver_path):
        self.finished = {}  # put in form link: mid_data_dd's
        self.links = links
        self.driver = webdriver.Chrome(executable_path=driver_path)
        self.driver.get(login_link)
        time.sleep(4)
        self.logIn()
        self.soup = None
    
    def getURL(self, profile_url):
        self.driver.get(profile_url)
        start = time.time()
        
        # the scrolling
        initialScroll = 0
        finalScroll = 1000
        while True:
            self.driver.execute_script(f"window.scrollTo({initialScroll},{finalScroll})")
            initialScroll = finalScroll
            finalScroll += 1000
            time.sleep(1.5)
            end = time.time()
            if round(end - start) > 8:
                break
        
        # SOURCE CODE
        src = self.driver.page_source
        # formatted
        soup = BeautifulSoup(src, 'html.parser')
        self.soup = soup   # just for lolo
        # by tag
        span = soup.find_all('span')
        #p = soup.find_all('p')
        
        #print(f"\n{profile_url}:")
        # filter/extracting data from raw profile data
        mid_dd = editSpan(span)
        self.finished[profile_url] = mid_dd


In [63]:
#### TIME
scrapper = LinkedOut([], driverPath)

C:\Users\Bill\AppData\Local\Temp\ipykernel_40068\1530215900.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(executable_path=driver_path)


In [ ]:
link1 = "https://www.linkedin.com/in/username/"
scrapper.getURL(link1)

In [65]:
wordsIG = scrapper.soup.findAll("span", attrs={"aria-hidden": "true"})
wordsIG = list(wordsIG)

for i in range(len(wordsIG)):
    item = str(wordsIG[i])
    if item.count("<!-- -->") >= 2:
        item_clean = re.sub("<(.)+?>", "", item)
        if item_clean in Segments:
            print(f"\n{item_clean}")
        else:
            print(" " * 4, item_clean)

     45 mutual connections: Stephen Pijanowski, Jane Chiu, and 43 others

About
     Junior @UChicago from Singapore, Shanghai, and Indonesia; Interested in data science and product management.

Activity
     1,585 followers
     Chloe Tan commented on a post • 2w

Experience
     Product Management Intern
     Intuit · Internship
     Apr 2022 - Present · 3 mos
     Mountain View, California, United States
     Content Creator
     YouTube · Part-time
     Dec 2019 - Present · 2 yrs 7 mos
     • Plans, executes, and produces weekly videos on platform @itschloetan• Featuring 114,800 subscribers and 7M+ views as of June 2022
     Market Data Analyst Intern
     Citadel · Internship
     Jan 2022 - Mar 2022 · 3 mos
     Chicago, Illinois, United States
     • Compiled financial records on traditional and alternative data; Generated databased and cleaned data for analysis• Wrote Excel scripts to automate advanced pivot tables; Optimized workflow efficiency by 70% (duration)• Streamlined m

In [50]:

resume_data = list(scrapper.soup.findAll("span", attrs={"aria-hidden": "true"}))
dd_resume_data = {}

active_segment = ""
for s in resume_data:
    s = str(s)
    if s.count("<!-- -->") >= 2:
        s_clean = re.sub("<(.)+?>", "", s)
        
        # if its a core area like "Experience"
        if s_clean in Segments:
            # check as to not rewrite hash
            active_segment = s_clean
            if active_segment not in dd_resume_data:
                dd_resume_data[active_segment] = []
        elif active_segment:
            dd_resume_data[active_segment] += [s_clean]
        
        #if active_segment not in dd_resume_data:
        #    dd_resume_data[active_segment] = []
        
        #else:
         #   dd_resume_data[active_segment] = [s_clean]
  

In [ ]:
for item in dd_resume_data:
    print(item)
    for entry in dd_resume_data[item]:
        print(entry)
    print("")
    #print(dd_resume_data[item], "\n\n")

# s_tt  
eh too much stuff

In [48]:
s_tt = re.sub("(\n)+", "\n", scrapper.soup.text)
s_tt = s_tt.split("\n")
s_tt = [s for s in s_tt if s and not all([l == " " for l in s])]
s_tt = [re.sub("( )+", " ", s) for s in s_tt]

In [26]:
text = re.sub("(\n)+", " \n", scrapper.soup.text)

In [61]:
wordsIG = scrapper.soup.findAll("span", attrs={"aria-hidden": "true"})
wordsIG = list(wordsIG)

for i in range(len(wordsIG)):
    item = str(wordsIG[i])
    if item.count("<!-- -->") >= 2:
        item_clean = re.sub("<(.)+?>", "", item)
        if item_clean in Segments:
            print(f"\n{item_clean}")
        else:
            print(" " * 4, item_clean)

     45 mutual connections: Stephen Pijanowski, Jane Chiu, and 43 others

About
     Junior @UChicago from Singapore, Shanghai, and Indonesia; Interested in data science and product management.

Activity
     1,585 followers
     Chloe Tan commented on a post • 2w

Experience
     Product Management Intern
     Intuit · Internship
     Apr 2022 - Present · 3 mos
     Mountain View, California, United States
     Content Creator
     YouTube · Part-time
     Dec 2019 - Present · 2 yrs 7 mos
     • Plans, executes, and produces weekly videos on platform @itschloetan• Featuring 114,800 subscribers and 7M+ views as of June 2022
     Market Data Analyst Intern
     Citadel · Internship
     Jan 2022 - Mar 2022 · 3 mos
     Chicago, Illinois, United States
     • Compiled financial records on traditional and alternative data; Generated databased and cleaned data for analysis• Wrote Excel scripts to automate advanced pivot tables; Optimized workflow efficiency by 70% (duration)• Streamlined m